# Eliminação Recursiva de Atributos (RFE) - Experimento

Esse componente executa a classificação de atributos com eliminação recursiva de atributos com base em um estimador do Random Forest com hiperparâmetros padrão. A validação cruzada, K-fold, é empregada para estimar a importância do recurso. Faz uso da implementação `RFECV` do [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para montagem de modelo, pré-processamento de dados, seleção e avaliação de modelos, e muitos outros utilitários.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "/tmp/data/boston.csv" #@param {type:"string"}
target = "medv" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

min_features = 3 #@param {type:"number", label: "Número mínimo de features a ser selecionado"}
n_folds = 10 #@param {type:"number", label: "Número de folds para a validação cruzada"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)
X = df.drop(target, axis=1)
y = df[target]

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
target_type = featuretypes[target_index]

columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Define a ordem de cada atributo

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# After the step handle_missing_values,
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = np.arange(len(numerical_indexes))
non_numerical_indexes_after_handle_missing_values = np.arange(
    len(numerical_indexes), len(featuretypes)
)

## Treina um seletor de atributos usando sklearn.feature_selection.RFECV

In [ ]:
from category_encoders.ordinal import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

if target_type[0] == NUMERICAL:
    estimator = RandomForestRegressor(random_state=0)
else:
    estimator = RandomForestClassifier(random_state=0)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

pipeline = Pipeline(
    steps=[
        (
            "handle_missing_values",
            ColumnTransformer(
                [
                    ("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                    (
                        "imputer_mode",
                        SimpleImputer(strategy="most_frequent"),
                        non_numerical_indexes,
                    ),
                ],
                remainder="drop",
            ),
        ),
        (
            "handle_categorical_features",
            ColumnTransformer(
                [
                    (
                        "handle_cat_features",
                        OrdinalEncoder(),
                        non_numerical_indexes_after_handle_missing_values,
                    )
                ],
                remainder="passthrough",
            ),
        ),
        (
            "rfe_estimator",
            RFECV(estimator, min_features_to_select=min_features, cv=n_folds),
        ),
    ]
)

pipeline.fit(X, y)

## Atributos selecionados

In [ ]:
selected_features = np.array(
    columns[numerical_indexes].tolist() + columns[non_numerical_indexes].tolist()
)
selected_features = selected_features[pipeline["rfe_estimator"].support_].tolist()

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
# save dataset changes
df.to_csv(dataset, index=False)

## Salva modelo e outros resultados do treinamento

Escreve todos artefatos na pasta `/tmp/data/`. A plataforma guarda os artefatos desta pasta para usos futuros como implantação e comparação de resultados.

In [ ]:
from joblib import dump

artifacts = {
    "pipeline": pipeline,
    "selected_features": selected_features,
}

dump(artifacts, "/tmp/data/rfe-selector.joblib")